# 트리의 앙상블

* 앙상블 모델(ensemble learning) : 정형 데이터를 다루는 데 가장 뛰어난 성과를 내는 알고리즘
* 대부분 결정 트리를 기반으로 만들어져 있음

* 정형 데이터
    * csv, 엑셀, json, xml 등 구조화 된 방식으로 된 데이터
* 비정형 데이터
    * 이미지, 영상, 텍스트, 오디오

### 랜덤포레스트 (random forest)
* 앙상블 학습의 대표 주자 중 하나로 안정적인 성능 덕분에 널리 사용됨
* 결정 트리를 랜덤하게 만들어 나무의 숲을 만든다 
* 그리고, 각 결정 트리의 예측을 사용해 최종 예측 